In [88]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Keys, ActionChains
import requests
import numpy as np

In [89]:
from selenium import webdriver

driver=webdriver.Chrome()

driver.get("https://absite.ru/sudoku/")

In [90]:
cells = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table.sudoku td"))
)

grid = np.zeros((9, 9), dtype=int)

for i, cell in enumerate(cells):
    row = i // 9
    col = i % 9
    value = cell.text.strip()
    if value.isdigit():
        grid[row][col] = int(value)
    

In [91]:
grid

array([[0, 0, 0, 5, 6, 1, 0, 0, 0],
       [2, 0, 0, 7, 8, 0, 6, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [6, 0, 3, 0, 0, 0, 1, 0, 0],
       [4, 7, 0, 0, 0, 5, 0, 0, 8],
       [0, 5, 0, 0, 0, 0, 0, 4, 3],
       [0, 0, 4, 8, 2, 0, 0, 0, 7],
       [0, 0, 7, 0, 0, 0, 0, 0, 0],
       [0, 8, 0, 0, 1, 0, 0, 3, 0]])

In [92]:
def findNextCellToFill(grid, i, j):
        for x in range(i,9):
                for y in range(j,9):
                        if grid[x][y] == 0:
                                return x,y
        for x in range(0,9):
                for y in range(0,9):
                        if grid[x][y] == 0:
                                return x,y
        return -1,-1

def isValid(grid, i, j, e):
        rowOk = all([e != grid[i][x] for x in range(9)])
        if rowOk:
                columnOk = all([e != grid[x][j] for x in range(9)])
                if columnOk:
                        # finding the top left x,y co-ordinates of the section containing the i,j cell
                        secTopX, secTopY = 3 *(i//3), 3 *(j//3) #floored quotient should be used here. 
                        for x in range(secTopX, secTopX+3):
                                for y in range(secTopY, secTopY+3):
                                        if grid[x][y] == e:
                                                return False
                        return True
        return False

def solveSudoku(grid, i=0, j=0):
        i,j = findNextCellToFill(grid, i, j)
        if i == -1:
                return True
        for e in range(1,10):
                if isValid(grid,i,j,e):
                        grid[i][j] = e
                        if solveSudoku(grid, i, j):
                                return True
                        # Undo the current cell for backtracking
                        grid[i][j] = 0
        return False

In [93]:
solveSudoku(grid)

True

In [94]:
grid

array([[7, 4, 9, 5, 6, 1, 3, 8, 2],
       [2, 3, 5, 7, 8, 4, 6, 9, 1],
       [8, 1, 6, 3, 9, 2, 7, 5, 4],
       [6, 2, 3, 9, 4, 8, 1, 7, 5],
       [4, 7, 1, 2, 3, 5, 9, 6, 8],
       [9, 5, 8, 1, 7, 6, 2, 4, 3],
       [3, 6, 4, 8, 2, 9, 5, 1, 7],
       [1, 9, 7, 4, 5, 3, 8, 2, 6],
       [5, 8, 2, 6, 1, 7, 4, 3, 9]])

In [95]:
for row in grid:
    for col in row:
        ActionChains(driver)\
            .sendKeys(f'{col}')\
            .key_down(Keys.ARROW_RIGHT)\
            .perform()

AttributeError: 'ActionChains' object has no attribute 'sendKeys'